In [ ]:
# Wrangling/math/stats
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score, accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# System utils
import os
from pathlib import Path

# Utils
from utils.consts import APPRAISAL_COLS, PURCHASE_COLS
from utils.imputation_performance import report_imputation_performance
from utils.regression_imputation import impute_by_regression

# Get local file structure
CWD = os.getcwd()
CWD = Path(CWD)
DATA = CWD / "data"
MODEL_DATA = DATA / "data_for_modeling"
MODEL_DATA.mkdir(parents=True, exist_ok=True)

# Imputation of Missing Data

In Part 1 I showed how many values are missing (close to 1/3rd of rows have at least one value missing). Given this, I plan to impute the missing data.

## Simple Imputation
For simplicity, I'm mostly going to use means/modes as much as possible to fill in missing data. Generally, I'll do this by making information decisions on how to subset the data. For instnce, using the mean or mode for a value ater subsetting on make/model.

## (More) Complex Imputation
After the initial imputation, I'll perform some more complex imputation using things like logistic and linear regression, primarily to fill in really commonnly missing columns:
* trim_descrip (premium vs non-premium trim) using logistic regression
* body using mutinomial regression

A better approach would be to use iterative imputation, but the implementation in sklearn currently doesn't support the wide array of complex variable types in this dataset. For two variables, I will settle on classifier models that make use of only the naive/simple imputations for training.

## Assessing Accuracy of Imputed Data
In order to avoid information leaking (i.e. imputation capturing information about the testing data and inflating test performance), I'll define a test/train split and use only the training data for imputation and use the non-missing values in the test set to assess performance.

Lastly, I'll keep track of which rows/columns have imputed values so I can evaluate how this may impacted my final value regression model(s) later.

When I train regressors in future notebooks, I plan to train in two ways: with imputed data and without imputed data (i.e. dropping columns with missing data).

In both cases I'll report model performance on three subsets of the test and training data separately:
* imputed data + complete data overall
* complete data only
* imputed data only

This will give a sense of how helpful the imputation was for model performance and whether imputed and non-imputed performance is comparable.

Since there are so many rows missing just a single value, I expect that even fairly naive, somewhat ill-informed imputations (i.e. column mean or mode) will help just by increasing the amount of training data available to learn generalizable relationships.

When performing model inference later, I'll also make sure to 

In [ ]:
kmax_data = pd.read_pickle(DATA/"long_kmax.pkl")
train_data, test_data = train_test_split(kmax_data, test_size=.15)
train_data["split"] = "train"
test_data["split"] = "test"
kmax_data = pd.concat([train_data, test_data])
kmax_data["market"] = kmax_data["market"].astype(str)
kmax_data

#### Working with combined appraisal + purchase data
Some models are rare and half are in both data sets (shown in previous dataset), and trends regarding other aspects of car features are likely to hold whether the car is being sold or purchases, so I'll use vehicle information from both sales and purchases alike to impute values in most cases.

For this I converted the data to long form where each row is one end of a transaction and there is an indicator for whether it was an appraisal or purchase.
Note: Carmax confirmed that the obfuscated make/model labels are the same for across appraisals and purchases.

In [ ]:
def report_missing_vals(df: pd.DataFrame):
    """
    Prints number of rows within specified columns with any missing values and returns
    table summarizing which rows have missing values.

    Args:
        df (pandas.DataFrame): The input data to check for missing values
        subset (str): A string indicating which subset of columns to check: 'purchase,'
            'appraisal,' or 'all'
    """
    subset = "all"
    if subset == "purchase":
        cols = PURCHASE_COLS
    elif subset == "appraisal":
        cols = APPRAISAL_COLS
    elif subset == "all":
        cols = df.columns

    rows_with_missing = df[cols].isna().any(axis=1).sum()
    print(f"{rows_with_missing} rows with missing ({subset} columns) data remain.")

    sns.histplot(df[cols].isna().sum(axis=1))
    plt.title(f"Missing Values ({subset.capitalize()} Columns)");
    plt.show()
    plt.close()
    
    return df[cols].isna().sum().sort_values(ascending=False)

#### Overall NAs

In [ ]:
report_missing_vals(kmax_data)

#### Training NAs

In [ ]:
report_missing_vals(kmax_data[kmax_data["split"] == "train"])

#### Testing NAs

In [ ]:
report_missing_vals(kmax_data[kmax_data["split"] == "test"])

# Initial Imputation Pass

In [ ]:
from typing import Optional
def update_imputed_vals(
        df: pd.DataFrame,
        col: str,
        imputed_vals: np.ndarray,
        imputed_vals_mask: Optional[np.ndarray] = None
        ):
    """_summary_

    Parameters
    ----------
    df : pd.DataFrame
        _description_
    col : str
        _description_
    imputed_vals : np.ndarray
        _description_
    imputed_vals_mask : np.ndarray
        _description_
    """
    imp_indicator_col = f"{col}_imputed"
    has_imp_col = imp_indicator_col not in df.columns
    no_explicit_imp_mask = isinstance(imputed_vals_mask, type(None))
    if not no_explicit_imp_mask:
        df[imp_indicator_col] = imputed_vals_mask  
    elif has_imp_col and no_explicit_imp_mask:
        imputed_vals_mask = df[imp_indicator_col]
    elif not has_imp_col and no_explicit_imp_mask:
        raise ValueError("Must pass imputed values mask if column does not already exist")

    df.loc[imputed_vals_mask, col] = imputed_vals[imputed_vals_mask]

## Impute online appraisal flag
Assume the purchase was offline if there is no flag.

It seems unlikely that online transactions have a missing trail, it's also more common in the dataset anyways.

I won't bother with imputing this iteratively (i.e. via a logistic regressor), since it's such a small number of values.

In [ ]:
kmax_data["online_appraisal_flag"].value_counts(dropna=False)/len(kmax_data)

In [ ]:
imputed_appraisal_flag_mask = kmax_data["online_appraisal_flag"].isna()
imputed_appraisal_flag = np.zeros(len(kmax_data))
report_imputation_performance(df=kmax_data, col="online_appraisal_flag", imputed_vals=imputed_appraisal_flag, metric=accuracy_score)
update_imputed_vals(kmax_data, "online_appraisal_flag", imputed_appraisal_flag, imputed_appraisal_flag_mask)

In [ ]:
kmax_data["online_appraisal_flag"].value_counts(dropna=False)/len(kmax_data)

In [ ]:
report_missing_vals(kmax_data)

### Impute colors
Here I'm just going to label it as unknown since there is already an "unknown" label. I'm doubtful that color is meaningfully encoded by other features.

In [ ]:
kmax_data["color"].value_counts()

In [ ]:
### Will just merge NA values into the existing "Unknown" column
kmax_data["color"] = kmax_data["color"].fillna("Unknown")

In [ ]:
kmax_data["color"].value_counts()

In [ ]:
report_missing_vals(kmax_data)

## Imputing Model
I'll impute model by assuming the most common in each make.

In [ ]:
# Get most common model for each make
most_common_models = train_data.groupby('make')['model'].agg(pd.Series.mode)
mode_models_by_make = most_common_models.to_dict()
for make_key, model_val in mode_models_by_make.items():
    if model_val and not isinstance(model_val, str):
        mode_models_by_make[make_key] = model_val[0]
### Fill based on most common appraisal model for each make
imputed_model_mask = kmax_data['model'].isna()
imputed_models = kmax_data['make'].map(mode_models_by_make)
report_imputation_performance(df=kmax_data, col="model", imputed_vals=imputed_models, metric=f1_score, plot=False)
update_imputed_vals(kmax_data, "model", imputed_models, imputed_model_mask)

This is not very good. Model is also a very high-dimension factor. It's also a very fleeting thing (a model is far more likely to stop existing in the future than a make). I may just ignore this as a variable in the future.

#### Check how much data is missing

In [ ]:
report_missing_vals(kmax_data)

## Imputing Body

I'll do this based primarily on model, filling anything that is still missing with the most common overall.
I would probably be better to train a classifier based on other characteristics after initially filling with the model, but since only purchases (which I am chosing not to mostly not make use of at the moment) have missing values I won't both.

In [ ]:
### Fill body based on the most common for that model
body_by_models = train_data.groupby("model")["body"].agg(pd.Series.mode)
body_by_models = body_by_models.to_dict()

body_freqs = train_data["body"].value_counts(sort=True).reset_index().rename(columns={"index":"body","body":"count"})

imputed_body_by_models = {}
for model, body in body_by_models.items():
    if isinstance(body, np.ndarray) or isinstance(body, str):
        if len(body) > 1:
            body_inds = body_freqs.index[body_freqs['body'].isin(body)]
            body = body_freqs.iloc[min(body_inds)]['body']
        else:
            body = np.nan
    else:
        body = np.nan
    imputed_body_by_models[model]  = body

imputed_body_mask = kmax_data['body'].isna()
imputed_body = kmax_data['body'].fillna(kmax_data['model'].map(imputed_body_by_models))
missing_model_body = imputed_body.isna()
imputed_body = imputed_body.fillna(kmax_data['body'].mode().values[0])
body_train_f1, body_test_f1 = report_imputation_performance(df=kmax_data, col="body", imputed_vals=imputed_body, metric=f1_score)
update_imputed_vals(df=kmax_data, col="body", imputed_vals=imputed_body, imputed_vals_mask=imputed_body_mask)

In [ ]:
report_missing_vals(kmax_data)

## MPG City/Highway & Fuel Efficiency
Note: MPG city/highway correlate really, really well, but they are both always missing at the same time (see cell immediately below).
Instead, I'll use model. Where model doesn't work, I'll fill based on body average.

### MPG
Note: MPG city/highway correlate really, really well, but they are both always missing at the same time (see cell immediately below).
Instead, I'll use model, body, and overall mean to help imput MPG.

In [ ]:
agreement = accuracy_score(kmax_data["mpg_city"].isna(), kmax_data["mpg_highway"].isna())
print(f"Agreement between missing MPG city and highay values is {agreement*100:.2f}%")

#### MPG City

In [ ]:
#### Impute MPG city
model_mpg_city = train_data.groupby("model")["mpg_city"].agg(pd.Series.mean).to_dict()
body_mpg_city = train_data.groupby("body")["mpg_city"].agg(pd.Series.mean).to_dict()
imputed_mpg_city = kmax_data["model"].map(model_mpg_city)
imputed_mpg_city = imputed_mpg_city.fillna(kmax_data["body"].map(body_mpg_city))
imputed_mpg_city = imputed_mpg_city.fillna(train_data['mpg_city'].mean())
imputed_mpg_city_mask = kmax_data['mpg_city'].isna()
mpg_city_train_r2, mpg_city_test_r2 = report_imputation_performance(df=kmax_data, col="mpg_city", imputed_vals=imputed_mpg_city, metric=r2_score)
update_imputed_vals(df=kmax_data, col="mpg_city", imputed_vals=imputed_mpg_city, imputed_vals_mask=imputed_mpg_city_mask)

#### MPG Highay

In [ ]:
#### Impute MPG highway
model_mpg_hwy = train_data.groupby("model")["mpg_highway"].agg(pd.Series.mean).to_dict()
body_mpg_hwy = train_data.groupby("body")["mpg_highway"].agg(pd.Series.mean).to_dict()
imputed_mpg_hwy = kmax_data["model"].map(model_mpg_hwy)
imputed_mpg_hwy = imputed_mpg_hwy.fillna(kmax_data["body"].map(body_mpg_hwy))
imputed_mpg_hwy = imputed_mpg_hwy.fillna(train_data['mpg_highway'].mean())
imputed_mpg_hwy_mask = kmax_data["mpg_highway"].isna()
mpg_hwy_train_r2, mpg_hwy_test_r2 = report_imputation_performance(df=kmax_data, col="mpg_highway", imputed_vals=imputed_mpg_hwy, metric=r2_score)
update_imputed_vals(kmax_data, "mpg_highway", imputed_mpg_hwy, imputed_mpg_hwy_mask)

### Fuel Capacity

In [ ]:
# Impute fuel capacity
model_fuel_capacity = train_data.groupby("model")["fuel_capacity"].agg(pd.Series.mean).to_dict()
body_fuel_capacity = train_data.groupby("body")["fuel_capacity"].agg(pd.Series.mean).to_dict()
mean_fuel_capacity = train_data["fuel_capacity"].mean()
imputed_fuel_capacity = kmax_data["body"].map(body_fuel_capacity)
imputed_fuel_by_body = kmax_data["model"].map(model_fuel_capacity)
imputed_fuel_capacity = imputed_fuel_capacity.fillna(kmax_data["model"].map(model_fuel_capacity))
imputed_fuel_capacity = imputed_fuel_capacity.fillna(mean_fuel_capacity)
# Get mask
imputed_fuel_capacity_mask = kmax_data["fuel_capacity"].isna()
# Check and update
fuel_train_r2, fuel_test_r2 = report_imputation_performance(kmax_data, "fuel_capacity", imputed_fuel_capacity, r2_score)
update_imputed_vals(kmax_data, "fuel_capacity", imputed_fuel_capacity, imputed_fuel_capacity_mask)

In [ ]:
report_missing_vals(kmax_data)

## Impute Trim
Fill in the most common trim type for each model (use imputed model--aka most common model for the make where not available)

In [ ]:
trim_freqs = train_data["trim_descrip"].value_counts(sort=True).reset_index().rename(columns={"index":"trim_descrip","trim_descrip":"count"})
### Fill trim based on the most common for that model
_trim_by_models = train_data.groupby("model")["trim_descrip"].agg(pd.Series.mode)
_trim_by_models = _trim_by_models.to_dict()
trim_by_models = {}
for model, trim in _trim_by_models.items():
    if isinstance(trim, np.ndarray):
        pass
    else:
        trim_by_models[model] = trim

# Trim descrip
imputed_trim = kmax_data['model'].map(trim_by_models)
imputed_trim = imputed_trim.fillna("Not Premium")
imputed_trim_mask = kmax_data['trim_descrip'].isna()
trim_train_acc, trim_test_acc = report_imputation_performance(df=kmax_data, col="trim_descrip", imputed_vals=imputed_trim, metric=accuracy_score)
update_imputed_vals(kmax_data, "trim_descrip", imputed_trim, imputed_trim_mask)

In [ ]:
report_missing_vals(kmax_data)

# Regression-Based Imputation Models
Using the initially imputed values I'm now going to fit a regression model for each factor using the others to predict it.

Three types of basic models will be used:
- Linear regression (multiple linear regression)
    - MPG City
    - MPG Highway
    - Fuel Capacity
- Logistic regression
    - Online appraisal flag
    - Trim description
- Multinomial Logistic Regression
    - Body type

I will not be performing any iterations -- each model will be fit based on known + imputed values from above.

I'll also be making use of "informed" model selection for fitting these. That is, obvservations I made during EDA and/or know about cars to select features. I'm not aiming for the perfect model here, just a quickly refined set of values.

In [ ]:
model_kmax_data = kmax_data.copy()

## MPG (Linear Regession)

In [ ]:
imputed_mpg_city2 = impute_by_regression(
    data=model_kmax_data,
    target_col="mpg_city",
    model_class=LinearRegression,
    metric=r2_score,
    report_kwargs={"previous_scores": (mpg_city_train_r2, mpg_city_test_r2)}
    )

In [ ]:
imputed_mpg_hwy2 = impute_by_regression(
    data=model_kmax_data,
    target_col="mpg_highway",
    model_class=LinearRegression,
    metric=r2_score,
    report_kwargs={"previous_scores": (mpg_hwy_train_r2, mpg_hwy_test_r2)}
    )

#### Updating values
I'm going to apply the linear model for mpg_highway, but not mpg_city.
mpg_highway is a really nice fit
mpg_city doesn't look much better, but since we seem to have a nice model for mpg_highway and mpg_city is so closely tied to mpg_highway, I'd like to perform the imputation iteratatively

In [ ]:
update_imputed_vals(kmax_data, "mpg_highway", imputed_mpg_hwy2, kmax_data["mpg_highway_imputed"])

### Re-try MPG city with updated MPG highway values.

In [ ]:
imputed_mpg_city3 = impute_by_regression(
    data=kmax_data,
    target_col="mpg_city",
    model_class=LinearRegression,
    metric=r2_score,
    report_kwargs={"previous_scores": (mpg_city_train_r2, mpg_city_test_r2)}
    )

How close are the regression imputed MPG city models before and after supplying updated MPG highway values?

In [ ]:
sns.scatterplot(x=imputed_mpg_city2[imputed_mpg_city_mask], y=imputed_mpg_city3[imputed_mpg_city_mask])
plt.xlabel("Trained before updating MPG Highway")
plt.ylabel("Trained after updating MPG Highway")
plt.title("Comparison of Imputed MPG City Values Across Models");

In [ ]:
update_imputed_vals(
    df=kmax_data,
    col="mpg_city",
    imputed_vals=imputed_mpg_city3,
    imputed_vals_mask=kmax_data["mpg_city_imputed"])

## Fuel Capacity (Linear Regession)

In [ ]:
imputed_fuel2 = impute_by_regression(
    data=model_kmax_data,
    target_col="fuel_capacity",
    model_class=LinearRegression,
    metric=r2_score,
    report_kwargs={"previous_scores": (fuel_train_r2, fuel_test_r2)}
    )

This is pretty obviously better.

In [ ]:
update_imputed_vals(
    df=kmax_data,
    col="fuel_capacity",
    imputed_vals=imputed_fuel2,
    imputed_vals_mask=kmax_data["fuel_capacity_imputed"])

## Trim Description (Logistic Regression)

In [ ]:
imputed_trim2 = impute_by_regression(
    data=model_kmax_data,
    target_col="trim_descrip",
    model_class=LogisticRegression,
    metric=accuracy_score,
    report_kwargs={"previous_scores": (trim_train_acc, trim_test_acc)},
    model_kwargs={"max_iter": 5000}
    )

Agreement between naive and regression-based imputed values

In [ ]:
model_agreement = imputed_trim == imputed_trim2
agreement_imputed = model_agreement[imputed_trim_mask]
print(f"Agreement between imputed values is {agreement_imputed.sum()/imputed_trim_mask.sum()}")

Comparing ratios of premium to not premium trim across different subsets of known and predicted alues.

In [ ]:
print("Ratios for known values")
cts_trima = kmax_data[~imputed_trim_mask]["trim_descrip"].value_counts()
print(cts_trima/cts_trima.sum())
print("Ratios for known values (where models disagree)")
cts_trimb = kmax_data[~imputed_trim_mask & ~model_agreement]["trim_descrip"].value_counts()
print(cts_trimb/cts_trimb.sum())

In [ ]:
cts_imputed1a = imputed_trim[imputed_trim_mask].value_counts()
cts_imputed1b = imputed_trim[imputed_trim_mask & ~model_agreement].value_counts()
print("Ratios for simple model")
print(cts_imputed1a/cts_imputed1a.sum())
print("Ratios for simple model (where models disagree)")
print(cts_imputed1b/cts_imputed1b.sum())

In [ ]:
cts_imputed2a = pd.Series(imputed_trim2)[imputed_trim_mask].value_counts()
cts_imputed2b = pd.Series(imputed_trim2)[imputed_trim_mask & ~model_agreement].value_counts()
print("Ratios for regression model")
print(cts_imputed2a/cts_imputed2a.sum())
print("Ratios for regression model (where models disagree)")
print(cts_imputed2b/cts_imputed2b.sum())

Looking at what other values are missing when imputed values disagree

In [ ]:
trim_imputed_segment = kmax_data[[c for c in kmax_data.columns if "imputed" in c]][imputed_trim_mask]
print("Missing data when both imputed values agree")
print(trim_imputed_segment[agreement_imputed].sum()/len(trim_imputed_segment[agreement_imputed]))
print()
print("Missing data when imputed values disagree")
print(trim_imputed_segment[~agreement_imputed].sum()/len(trim_imputed_segment[~agreement_imputed]))

### Choosing which imputed trim values to use
When they disagree the vehicle model is much less likely to be known, while the body is a bit more likely to be known.

Further, the ratios of premium to non-premium trim returned by the regression model line up a bit better with the average of known examples when they disagree.

Based on this and the overall comparable performance, I'm going to go with the regression imputed trim. This is also helpful since working with vehicle model is a bit difficult and I'd like to reduce reliance on it where possible since vehicle models get retired and introduced over time and this will impact the ability of my ML model to work in the future (even though it's a toy project it's still good to consider)

In [ ]:
update_imputed_vals(
    df=kmax_data,
    col="trim_descrip",
    imputed_vals=imputed_trim2,
    imputed_vals_mask=kmax_data["trim_descrip_imputed"])

## Body (Multinomial Logistic Regression)

In [ ]:
imputed_body2 = impute_by_regression(
    data=model_kmax_data,
    target_col="body",
    model_class=LogisticRegression,
    metric=f1_score,
    report_kwargs={"previous_scores": (body_train_f1, body_test_f1)},
    model_kwargs={"multi_class": "multinomial", "max_iter": 5000, "solver": "saga"}
    )

This performance is lower than using make, which isn't surprising, since make is extremely intricately linked with body. That said, the regression model has clearly learned some good relationships.

For some missing body values we imputed using the most common category because the model was either missing itself or had no available body data.

It would be better to instead use the regression model to impute these edge cases instead of just labeling them all the same, so I will use the multinomial logistic regression to fill-in where that was the case.

Agreement between simple (overall body) and MNLog body imputation methods:

In [ ]:
matching_missing_model_body = imputed_body2[missing_model_body] == imputed_body[missing_model_body]
missing_model_body_agreement = matching_missing_model_body.sum()/len(kmax_data)
print(f"The MNLog regression and mode of body generated {missing_model_body_agreement*100:.2f}% agreement for cases where the most common body per model was not known.")

Frequency of simple (overall mode) imputed body values (where model body info was not availabel)

In [ ]:
imputed_body[missing_model_body].value_counts()/missing_model_body.sum()

Frequency of MNLog imputed body values (where model body info was not available)

In [ ]:
pd.DataFrame({"body": imputed_body2[missing_model_body]})['body'].value_counts()/missing_model_body.sum()

#### Incorporate new body labels for the missing model-specific info

In [ ]:
kmax_data.loc[missing_model_body, "body"] = imputed_body2[missing_model_body]

In [ ]:
any_imputed_vals = kmax_data[[c for c in kmax_data.columns if "imputed" in c]].any(axis=1)
kmax_data["imputed"] = any_imputed_vals
app_data = kmax_data["itransaction"] == "appraisal"
kmax_data[app_data].drop(columns=["itransaction"]).to_pickle(MODEL_DATA/"appraisal.pkl")
kmax_data[~app_data].drop(columns=["itransaction"]).to_pickle(MODEL_DATA/"purchase.pkl")

### Update a few vals to make categorical/ordinal encoding easier

In [ ]:
from utils.preprocessing import get_midpoint
def merge_rare_cats(df: pd.DataFrame, cat: str, n: int = 20):
    train = df[df["split"]=="train"].copy()
    make_counts = train[cat].value_counts()
    non_rare_cats = make_counts[make_counts > n].index
    rare_cats = ~df[cat].isin(non_rare_cats)
    df.loc[rare_cats, cat] = "RARE"

files = [MODEL_DATA/"appraisal.pkl", MODEL_DATA/"purchase.pkl"]
for f in files:
    df = pd.read_pickle(f)
    df['mileage'] = get_midpoint(df['mileage']).rank(method="dense")
    df['value'] = get_midpoint(df['value']).rank(method="dense")
    train = df[df["split"]=="train"].copy()
    merge_rare_cats(df, "make")
    merge_rare_cats(df, "color")
    merge_rare_cats(df, "body")
    merge_rare_cats(df, "market")
    df.to_pickle(f)
    df.head()